In [1]:
import pymongo
import os
from faker import Faker
from bson import ObjectId
import random
from datetime import datetime, timedelta

# Initialize Faker with Spanish locale
faker = Faker('es_ES')

mongo_uri = os.getenv("MONGO_URI")

client = pymongo.MongoClient(mongo_uri)

# Select your database and collection
db = client['healthcare_workshop']  # Replace 'mydatabase' with your database name
collection = db['patients']  # Replace 'patients' with your collection name

def generate_email(first_name, last_name):
    # Create a basic email with first name and last name
    email_domains = ["gmail.com", "hotmail.com", "yahoo.com", "outlook.com"]
    email = f"{first_name.lower()}.{last_name.lower()}@{random.choice(email_domains)}"
    return email

def generate_emergency_contact(patient_lastname):
    # Randomly choose the relation and generate the appropriate name
    relation = random.choice(["parent", "sibling", "friend", "spouse"])
    
    if relation in ["parent", "sibling"]:
        # Use the same last name as the patient for parent/sibling
        emergency_first_name = faker.first_name()
        emergency_last_name = patient_lastname
    else:
        # For other relations, generate a completely random name
        emergency_first_name = faker.first_name()
        emergency_last_name = faker.last_name()

    emergency_name = f"{emergency_first_name} {emergency_last_name}"
    
    return {
        "name": emergency_name,
        "relation": relation,
        "phoneNumber": faker.phone_number()
    }

def generate_random_date_before_august_2024():
    # Generate a random date between January 1st and August 1st, 2024
    start_date = datetime(2024, 1, 1)
    end_date = datetime(2024, 8, 1)
    random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    return random_date

def generate_random_date_in_late_august_2024():
    # Generate a random date between August 2nd and August 31st, 2024
    start_date = datetime(2024, 8, 2)
    end_date = datetime(2024, 8, 31)
    random_date = start_date + timedelta(days=random.randint(0, (end_date - start_date).days))
    return random_date

def generate_fake_document():
    first_name = faker.first_name()
    last_name = faker.last_name()

    created_at = generate_random_date_before_august_2024()  # Random date between January 1st and August 1st, 2024
    updated_at = generate_random_date_in_late_august_2024()  # Random date between August 2nd and August 31st, 2024
    
    return {
        "_id": ObjectId(),  # Auto-generate ObjectId
        "firstName": first_name,
        "lastName": last_name,
        "birthday": faker.date_of_birth(minimum_age=0, maximum_age=100).strftime('%Y-%m-%d'),
        "gender": random.choice(["male", "female"]),
        "address": {
            "address1": faker.street_address(),
            "address2": "",  # Assuming no address2
            "postalCode": faker.postcode(),
            "city": faker.city(),
            "province": faker.state(),
            "country": "Spain"
        },
        "email": generate_email(first_name, last_name),  # Coherent email based on name and lastname
        "phoneNumber": [
            {
                "type": "mobile",
                "number": faker.phone_number()
            },
            {
                "type": "home",
                "number": faker.phone_number()
            }
        ],
        "emergencyContact": generate_emergency_contact(last_name),  # Coherent last name for parent/sibling
        "patientID": faker.bothify(text='PATIENT#########'),  # Random patient ID with numbers
        "createdAt": created_at.strftime('%Y-%m-%d'),  # Random date between January 1st and August 1st, 2024
        "updatedAt": updated_at.strftime('%Y-%m-%d')  # Random date between August 2nd and August 31st, 2024
    }

def insert_documents(num_docs):
    documents = [generate_fake_document() for _ in range(num_docs)]
    collection.insert_many(documents)
    print(f"Inserted {num_docs} documents into MongoDB.")

if __name__ == "__main__":
    # Generate and insert 10,000 documents
    insert_documents(10000)


Inserted 1 documents into MongoDB.
